In [1]:
cd /home/nakedmoleratvoices/Mole\ rats\ reborn/CodeRefactoredFinal/

/home/nakedmoleratvoices/Mole rats reborn/CodeRefactoredFinal


In [2]:
import numpy as np
import pandas as pd
import os
import pickle
import matplotlib.pyplot as plt
from src.preprocessing.process_sounds_metadata import generate_sounds_metadata, make_fixed_size_sounds
from src.preprocessing.filters import filter_recs_metadata
from src.preprocessing.load_data import load_recs_dict, load_sounds
from src.deep_learning.create_model import create_conv_model
from src.data_representations.process_wavs import *
from src.data_representations.process_images import *
%matplotlib inline


In [3]:
path_to_recordings_metadata = '/home/nakedmoleratvoices/Mole rats reborn/CodeRefactoredFinal/data/'
recordings_metadata_name = 'recordings_metadata.csv'

# what data to load
dates = 'all'
colonies = 'all'
experiments = 'all'
stages = ['traced and checked', 'traced', 'labeled and checked']

#?
goodrec = 'baratheon_20-04-20_4012_0000029.npy'
badrec = 'baratheon_11-04-20_2131_9460_0000023.npy'

#!
class_augment_dict = {'sound' : 3,  'noise' : 3}
sounds_max_length = 10000
sounds_min_length = 1337

In [4]:
# load metadata
recs_metadata = pd.read_csv(path_to_recordings_metadata + recordings_metadata_name)
mask = filter_recs_metadata(recs_metadata, dates, colonies, stages, experiments)
recs_for_training = recs_metadata[mask]
recs_dict = load_recs_dict(recs_for_training)
sounds_metadata = generate_sounds_metadata(recs_for_training).reset_index(drop = True)
print(sounds_metadata['cl'].value_counts())

# drop some softchirp for the sake of class balancing
max_softchirps = 22000
softchirps_inds = np.where(sounds_metadata['cl'] == 'softchirp')[0]
np.random.shuffle(softchirps_inds)
sounds_metadata = sounds_metadata.drop(softchirps_inds[max_softchirps :]).reset_index(drop = True)
sounds_metadata['cl'] = sounds_metadata['cl'].apply(lambda x : 'noise' if x == 'noise' else 'sound' )

lengths = 22050 * (sounds_metadata['e'] - sounds_metadata['s'])
mask_normal_sounds = (lengths >= sounds_min_length) & (lengths <= sounds_max_length)
sounds_metadata = sounds_metadata[mask_normal_sounds].reset_index(drop = True)
print(sounds_metadata['cl'].value_counts())

/usr/local/lib/python3.5/dist-packages/pandas/core/computation/expressions.py:193: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  op=op_str, alt_op=unsupported[op_str]


softchirp          68137
noise              42604
weirdo              7790
grunt               3275
loudchirp           2237
downsweep           1827
upsweep              718
whistle              524
combotwo             475
combo                367
scream               202
mordent               19
invertedmordent       10
vtrill                 6
Name: cl, dtype: int64
noise    39980
sound    36025
Name: cl, dtype: int64


In [ ]:
%%time
resolution = 1024 # ~0.05 sec
step = 512 # ~0.025 sec

sound_processing = {'stretching_lim_train' : None,
                    'stretching_lim_test' : None,
                    'noise_lim_train' : (0.001, 0.005),
                    'noise_lim_test' : None,
                    'filtering_th' : 3000
                   }
n_fft = 512
n_mel = 80

sounds_metadata_ints_split = make_fixed_size_sounds(sounds_metadata, resolution, step)
sounds_npy_split = load_sounds(sounds_metadata_ints_split, recs_dict, noisy_sampling = False, timestamps = 'int')
sounds_npy_pr_iterator = process_waves(sounds_npy_split, sound_processing['stretching_lim_train'],
                                       sound_processing['noise_lim_train'], sound_processing['filtering_th'])
melspecs_array = np.array(extract_melspecs(sounds_npy_pr_iterator, n_fft, n_mel))

img_shape = melspecs_array[0].shape
all_classes = ['noise', 'sound']
y_train_num = np.array([all_classes.index(yi) for yi in sounds_metadata_ints_split['cl']])

In [360]:
model = create_conv_model(img_shape, all_classes)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 80, 3, 256)        2560      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 40, 3, 256)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 40, 3, 256)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 40, 3, 256)        590080    
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 20, 3, 256)        0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 20, 3, 256)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 15360)            

In [362]:
model.fit(melspecs_array.reshape((*melspecs_array.shape, 1)), y_train_num, epochs = 2, batch_size = 256)

Epoch 1/5
1972/1972 [==============================] - 1683s 854ms/step - loss: 0.5124 - sparse_categorical_accuracy: 0.7480
Epoch 2/5
1972/1972 [==============================] - 1672s 848ms/step - loss: 0.3828 - sparse_categorical_accuracy: 0.8249
Epoch 3/5
1972/1972 [==============================] - 1670s 847ms/step - loss: 0.3637 - sparse_categorical_accuracy: 0.8358
Epoch 4/5
1972/1972 [==============================] - 1664s 844ms/step - loss: 0.3527 - sparse_categorical_accuracy: 0.8412
Epoch 5/5
1972/1972 [==============================] - 1683s 854ms/step - loss: 0.3470 - sparse_categorical_accuracy: 0.8438


In [363]:
model.save('./models/splitter/deep_splitter/80x1024.ckpt')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ./models/splitter/deep_splitter/80x1024.ckpt/assets
